<a href="https://colab.research.google.com/github/ulil-azmy/FMHYedit/blob/main/Release_Glints_Jobstreet_Kalibrr_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver

url = "https://glints.com/id/"

options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920,1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.get(url)

# Get cookies
cookies = driver.get_cookies()
print("Obtained cookies:")
for cookie in cookies:
    print(cookie)

driver.quit()


# **Release: Glints V1**

In [ ]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
import math
from datetime import datetime, timedelta

def fetch_jobs_data(offset):
    url = "https://glints.com/api/v2/graphql"
    headers = {
      "authority": "glints.com",
      "accept": "*/*",
      "accept-language": "id",
      "cache-control": "no-cache",
      "content-type": "application/json",
      "cookie": "builderSessionId=3014259a63494eafa39f092d3b7492f9",
      "origin": "https://glints.com",
      "pragma": "no-cache",
      "referer": "https://glints.com/id/opportunities/jobs/explore?keyword=%22data+analyst%22&country=ID&locationName=All+Cities%2FProvinces",
      "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
      "sec-ch-ua-mobile": "?0",
      "sec-ch-ua-platform": '"Windows"',
      "sec-fetch-dest": "empty",
      "sec-fetch-mode": "cors",
      "sec-fetch-site": "same-origin",
      "traceparent": "00-06c7ab8d4d8ccc81ec0e4cdb7a815b74-20420b7cb0cae343-01",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    }

    data = {
      "operationName": "searchJobs",
        "variables": {
        "data": {
            "SearchTerm": "\"data analyst\"",
            "CountryCode": "ID",
            "limit": 30,
            "sortBy": "LATEST",
            "offset": 0,
            "includeExternalJobs": True,
            "sources": ["NATIVE", "SUPER_POWERED"],
            "searchVariant": "VARIANT_A"
        }
    },
    "query": "query searchJobs($data: JobSearchConditionInput!) { searchJobs(data: $data) { jobsInPage { id title isRemote status createdAt updatedAt isActivelyHiring isHot shouldShowSalary educationLevel type fraudReportFlag salaryEstimate { minAmount maxAmount CurrencyCode } company { ...CompanyFields } citySubDivision { id name } city { ...CityFields } country { ...CountryFields } salaries { ...SalaryFields } location { ...LocationFields } minYearsOfExperience maxYearsOfExperience source type hierarchicalJobCategory { id level name children { name level id __typename } parents { id level name __typename } __typename } skills { skill { id name __typename } mustHave __typename } __typename } numberOfJobsCreatedInLast14Days totalJobs __typename } } fragment CompanyFields on Company { id name logo status IndustryId __typename } fragment CityFields on City { id name __typename } fragment CountryFields on Country { code name __typename } fragment SalaryFields on JobSalary { id salaryType salaryMode maxAmount minAmount CurrencyCode __typename } fragment LocationFields on HierarchicalLocation { id name administrativeLevelName formattedName level slug parents { id name administrativeLevelName formattedName level slug parents { level formattedName slug __typename } __typename } __typename }"
    }

    data['variables']['data']['offset'] = offset

    response = requests.post(url, headers=headers, data=json.dumps(data))
    response_json = response.json()
    jobs_in_page = response_json.get("data", {}).get("searchJobs", {}).get("jobsInPage", [])
    total_jobs = response_json.get("data", {}).get("searchJobs", {}).get("totalJobs", 0)

    return jobs_in_page, total_jobs

def generate_metadata(title):
    if not title:
        return ""
    metadata = title.lower().replace(" & ", "-and-")
    metadata = metadata.replace(" ", "-").replace("--", "-")
    return metadata

def construct_url(metadata, job_id):
    return f"https://glints.com/id/opportunities/jobs/{metadata}/{job_id}"

def calculate_dynamic_time_difference(updated_at):
    # Convert the updatedAt string to a datetime object
    updated_at_datetime = datetime.strptime(updated_at, "%Y-%m-%dT%H:%M:%S.%fZ")

    # Get the current datetime
    current_datetime = datetime.utcnow()

    # Calculate the time difference
    time_difference = current_datetime - updated_at_datetime

    # Calculate time difference in days, hours, and weeks
    days_difference = time_difference.days
    hours_difference = time_difference.seconds // 3600
    weeks_difference = days_difference // 7

    if weeks_difference >= 1:
        return f"{weeks_difference} week{'s' if weeks_difference > 1 else ''} ago"
    elif days_difference >= 1:
        return f"{days_difference} day{'s' if days_difference > 1 else ''} ago"
    elif hours_difference >= 1:
        return f"{hours_difference} hour{'s' if hours_difference > 1 else ''} ago"
    else:
        return "Less than an hour ago"


def generate_job_records(jobs_data):
    job_records = []
    for job in jobs_data:
        company_name = job.get("company", {}).get("name")
        city_name = job.get("city", {}).get("name")
        status = job.get("company", {}).get("status")
        updated_at = job.get("updatedAt")
        title = job.get("title")
        job_id = job.get("id")

        metadata = generate_metadata(title)
        url = construct_url(metadata, job_id)
        # Calculate dynamic time difference
        time_difference = calculate_dynamic_time_difference(updated_at)

        job_record = {
            "Company Name": company_name,
            "City Name": city_name,
            "Status": status,
            "updatedAt": updated_at,
            "Title": title,
            "URL": url,
            "Time Difference": time_difference  # Include the dynamic time difference in a new column
        }
        job_records.append(job_record)
    return job_records

def display_jobs_dataframe(job_records):
    df = pd.DataFrame(job_records)
    df['URL'] = df['URL'].apply(lambda url: f'<a href="{url}" target="_blank">{url}</a>')

    # Selecting only the first 5 and last 5 records
    top_and_bottom_records = pd.concat([df.head(5), df.tail(5)])

    styled_df = (
        top_and_bottom_records.style
        .set_table_styles([
            {
                'selector': 'th',
                'props': [
                    ('background-color', '#f5f5f5'),  # Lighter gray for header background
                    ('color', '#333333'),  # Darker gray for text color
                    ('font-weight', 'bold'),
                    ('padding', '8px'),
                    ('white-space', 'nowrap'),
                    ('border', '1px solid #e0e0e0'),  # Thinner border for header
                    ('font-family', 'Arial'),  # Different font family for table
                    ('font-size', '12px')  # Smaller font size for table
                ]
            },
            {
                'selector': 'tr',
                'props': [
                    ('background-color', '#ffffff'),
                    ('padding', '8px'),
                    ('border', '1px solid #e0e0e0'),  # Thinner border for rows
                    ('font-family', 'Arial'),
                    ('font-size', '12px')
                ]
            },
             {
                'selector': 'td',
                'props': [
                    ('white-space', 'nowrap'),
                    ('border', '1px solid #e0e0e0'),  # Thinner border for cells
                    ('font-family', 'Arial'),
                    ('font-size', '12px')
                ]
            },
            {
                'selector': 'tr:nth-child(even)',
                'props': [('background-color', '#f9f9f9')]
            },
            {
                'selector': 'tr:hover',
                'props': [('background-color', '#e0e0e0')]
            }
        ])

        .set_table_attributes('style="border-collapse: collapse; width: 100%;"')  # Setting table width
    )
    display(HTML(styled_df.to_html(escape=False)))
     #Save entire DataFrame as JSON
    df.to_json('glints_job.json', orient='records')  # Save DataFrame as JSON file

def main():
    jobs_data, total_count = fetch_jobs_data(0)
    print(f"Total job count: {total_count}")

    total_pages = math.ceil(total_count / 30)
    print(f"Total pages: {total_pages}")

    job_records = []
    for page in range(total_pages):
        offset = page * 30
        #print(f"Fetching jobs with offset: {offset} and limit: 30", end=' ')

        page_jobs, _ = fetch_jobs_data(offset)

        first_index = offset
        last_index = offset + len(page_jobs) - 1

        #print(f"| First index: {first_index} | Last index: {last_index}")
        # Log API response details
        #print(f"API Response: Limit: 30, Offset: {offset}, First index: {first_index}, Last index: {last_index}")
        job_records.extend(generate_job_records(page_jobs))
        # Save DataFrame as JSON
        #df.to_json('glints_job.json', orient='records')  # Save DataFrame as JSON file
    display_jobs_dataframe(job_records)

if __name__ == "__main__":
    main()

Total job count: 1000
Total pages: 34


# **Release: Jobstreet V1**

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch job data from a given URL
def fetch_job_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Error occurred: {e}")
        return None

# Function to format the listing date
def format_date(date_string):
    try:
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%SZ")
        return date_object.strftime("%d/%m/%Y %H:%M:%S")
    except ValueError:
        return 'N/A'

# Function to calculate time difference from the listing date to the current date
def calculate_time_difference(date_string):
    try:
        listing_date = datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S")
        current_date = datetime.now()

        time_difference = current_date - listing_date

        if time_difference < timedelta(days=1):
            return f"{int(time_difference.total_seconds() / 3600)} hours"
        elif time_difference < timedelta(days=7):
            return f"{int(time_difference.days)} days"
        else:
            return f"{int(time_difference.days / 7)} weeks"
    except ValueError:
        return 'N/A'

# Function to extract job details from each entry in the fetched data
def extract_job_details(entry):
    job_id = entry.get('id', 'N/A')
    formatted_date = format_date(entry.get('listingDate', 'N/A'))
    time_difference = calculate_time_difference(formatted_date)
    return {
        "Job_Title": entry.get('title', 'N/A'),
        "Company": entry['advertiser'].get('description', 'N/A'),
        "Location": entry.get('locationWhereValue', 'N/A'),
        "Posted_On": formatted_date,
        "Time_Elapsed": time_difference,
        "Salary": entry.get('salary', 'N/A'),
        "Apply_Here": f"https://www.jobstreet.co.id/id/job/{job_id}"
    }

# Function to fetch job data from multiple pages and extract details
def fetch_all_job_data(base_url, params, total_pages):
    all_data = []
    for page in range(1, total_pages + 1):  # Iterate through all pages
        params['page'] = str(page)
        url = base_url + "&".join([f"{key}={value}" for key, value in params.items()])
        print(f"Fetching data for page {page}: {url}")

        parsed_json = fetch_job_data(url)
        if parsed_json:
            for entry in parsed_json.get('data', []):
                if 'advertiser' in entry and 'description' in entry['advertiser']:
                    job_details = extract_job_details(entry)
                    all_data.append(job_details)
    return all_data

# Function to display a snippet of job data and save it to a JSON file
def display_and_save_job_data(all_data):
    df = pd.DataFrame(all_data)
    df['Number'] = df.reset_index().index + 1  # Adding a numbering column
    styled_df = pd.concat([df.head(10), df.tail(10)])
    styled_df = styled_df.style.set_properties(**{'text-align': 'left'})
    styled_df.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    display(styled_df)

    # Save DataFrame to a single properly formatted JSON file
    df.to_json('JobData.json', orient='records', indent=2)

# Function to fetch metadata including total job count and page size
def fetch_metadata(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            metadata = data.get('solMetadata', {})
            total_job_count = metadata.get('totalJobCount', 0)
            page_size = metadata.get('pageSize', 0)
            return total_job_count, page_size
        else:
            print(f"Failed to fetch metadata. Status code: {response.status_code}")
            return 0, 0
    except requests.RequestException as e:
        print(f"Error occurred: {e}")
        return 0, 0

# Function to calculate the number of pages based on total job count and page size
def calculate_total_pages(total_jobs, page_size):
    return (total_jobs + page_size - 1) // page_size

# Function to get total job count, page size, and number of pages
def get_job_information(base_url, params):
    metadata_url = base_url + "&".join([f"{key}={value}" for key, value in params.items()])
    total_jobs, page_size = fetch_metadata(metadata_url)
    total_pages = calculate_total_pages(total_jobs, page_size)
    return total_jobs, page_size, total_pages

# Main function orchestrating the process
def main():
    base_url = "https://www.jobstreet.co.id/api/chalice-search/v4/search?"
    params = {
        "siteKey": "ID-Main",
        "keywords": "%22data+analyst%22",
        "sortmode": "ListedDate",
        "include": "seodata",
        "locale": "id-ID",
    }

    total_jobs, page_size, total_pages = get_job_information(base_url, params)

    all_job_data = fetch_all_job_data(base_url, params, total_pages)
    display_and_save_job_data(all_job_data)

if __name__ == "__main__":
    main()


# **Release: Kalibrr V1**

In [ ]:
import requests
from urllib.parse import urlencode
import json
homepage = "https://www.kalibrr.id/id-ID/c/"

def log_endpoints(base_url, params):
    all_endpoints = []
    limit = params['limit']
    total_count = fetch_total_count(base_url, params)

    if total_count is not None:
        for offset in range(0, total_count, limit):
            params['offset'] = offset
            constructed_endpoint = construct_endpoint(base_url, params)
            all_endpoints.append(constructed_endpoint)

    return all_endpoints

def fetch_total_count(base_url, params):
    try:
        count_params = params.copy()
        count_params['limit'] = 1
        count_params['offset'] = 0

        total_count_url = construct_endpoint(base_url, count_params)
        #print(f"Total Count Endpoint URL: {total_count_url}")

        response = requests.get(base_url, params=count_params)
        response.raise_for_status()  # Raises an HTTPError for non-200 status codes

        total_count = response.json().get('count', 0)
        return total_count
    except requests.RequestException as e:
        print(f"Request Exception: {e}")
        return None

def construct_endpoint(base_url, params):
    encoded_params = urlencode(params)
    return f"{base_url}?{encoded_params}"  # Add the return statement to return the constructed URL

def fetch_data_from_endpoints(endpoints):
    job_data = []
    number = 1  # Initialize the numbering variable
    for endpoint in endpoints:
        try:
            response = requests.get(endpoint)
            if response.status_code == 200:
                data = response.json()
                jobs = data.get('jobs', [])

                for job in jobs:
                    job_info = {
                        'number': number,
                        'job_title': job.get('name'),
                        'company_name': job.get('company_name'),
                        'activation_date': job.get('activation_date'),
                        'city': job.get('google_location', {}).get('address_components', {}).get('city'),
                        'url': f"{homepage}{job['company']['code']}/jobs/{job['id']}/{job['slug']}"
                    }
                    job_data.append(job_info)
                    number += 1  # Increment the number for each job
            else:
                print(f"Failed with status code: {response.status_code}")

        except requests.RequestException as e:
            print(f"Request Exception: {e}")

    return job_data


base_url = "https://www.kalibrr.com/kjs/job_board/search"
params = {
    'limit': 15,
    'country': 'Indonesia',
    'sort_direction': 'asc',
    'sort_field': 'application_end_date',
    'text': 'Data-Analyst'
}

all_endpoints = log_endpoints(base_url, params)
job_info = fetch_data_from_endpoints(all_endpoints)

# Save the fetched job information into a JSON file
output_file = 'job_info.json'
with open(output_file, 'w') as file:
    json.dump(job_info, file, indent=4)

print(f"Job information saved to '{output_file}' successfully.")


# Print all the constructed endpoint URLs
#or endpoint in all_endpoints:
    #print(endpoint)


Job information saved to 'job_info.json' successfully.


# **DEBUG DEBUG**

In [ ]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json('job_info.json')

# Set display width (optional)
pd.set_option('display.width', 1000)  # Change the value as needed

# Style the DataFrame
styled_df = pd.concat([df.head(10), df.tail(10)])
styled_df = styled_df.style.set_properties(**{'text-align': 'left'})
styled_df.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])

# Display the styled DataFrame
display(styled_df)

# Print the original DataFrame
#print(df)


# **UNIFIEED IT ALL text**

In [134]:
import requests

# Configuration for each job portal
job_portals = [
{
    "name": "Glints",
    "url": "https://glints.com/api/v2/graphql",
    "headers": {
      "authority": "glints.com",
      "accept": "*/*",
      "accept-language": "id",
      "cache-control": "no-cache",
      "content-type": "application/json",
      "cookie": "builderSessionId=3014259a63494eafa39f092d3b7492f9",
      "origin": "https://glints.com",
      "pragma": "no-cache",
      "referer": "https://glints.com/id/opportunities/jobs/explore?keyword=%22data+analyst%22&country=ID&locationName=All+Cities%2FProvinces",
      "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
      "sec-ch-ua-mobile": "?0",
      "sec-ch-ua-platform": '"Windows"',
      "sec-fetch-dest": "empty",
      "sec-fetch-mode": "cors",
      "sec-fetch-site": "same-origin",
      "traceparent": "00-06c7ab8d4d8ccc81ec0e4cdb7a815b74-20420b7cb0cae343-01",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    },
    "data": {
        "operationName": "searchJobs",
        "variables": {
            "data": {
                 "SearchTerm": "\"data analyst\"",
                  "CountryCode": "ID",
                  "limit": 1,
                  "sortBy": "LATEST",
                  "offset": 0,
                  "includeExternalJobs": True,
                  "sources": ["NATIVE", "SUPER_POWERED"],
                  "searchVariant": "VARIANT_A"
            }
        },
       "query": "query searchJobs($data: JobSearchConditionInput!) { searchJobs(data: $data) { jobsInPage { id title isRemote status createdAt updatedAt isActivelyHiring isHot shouldShowSalary educationLevel type fraudReportFlag salaryEstimate { minAmount maxAmount CurrencyCode } company { ...CompanyFields } citySubDivision { id name } city { ...CityFields } country { ...CountryFields } salaries { ...SalaryFields } location { ...LocationFields } minYearsOfExperience maxYearsOfExperience source type hierarchicalJobCategory { id level name children { name level id __typename } parents { id level name __typename } __typename } skills { skill { id name __typename } mustHave __typename } __typename } numberOfJobsCreatedInLast14Days totalJobs __typename } } fragment CompanyFields on Company { id name logo status IndustryId __typename } fragment CityFields on City { id name __typename } fragment CountryFields on Country { code name __typename } fragment SalaryFields on JobSalary { id salaryType salaryMode maxAmount minAmount CurrencyCode __typename } fragment LocationFields on HierarchicalLocation { id name administrativeLevelName formattedName level slug parents { id name administrativeLevelName formattedName level slug parents { level formattedName slug __typename } __typename } __typename }"
    },
    "structure": {
            "total_jobs_key": "data.searchJobs.totalJobs",
            "number_of_pages_key": 1,
            "limit_key": 1,
            "offset_key": 0
        }
}
,
    {
       "name": "Jobstreet",
        "url": "https://www.jobstreet.co.id/api/chalice-search/v4/search",
        "params": {
            "siteKey": "ID-Main",
            "sourcesystem": "houston",
            "userqueryid": "309177c185dc2dfb91aa10ca597438d9-3746302",
            "userid": "e1073b62-7a54-4ee5-b5a9-2a3ec516d203",
            "usersessionid": "e1073b62-7a54-4ee5-b5a9-2a3ec516d203",
            "eventCaptureSessionId": "e1073b62-7a54-4ee5-b5a9-2a3ec516d203",
            "page": 1,
            "seekSelectAllPages": True,
            "keywords": "\"data+analyst\"",
            "pageSize": 1,
            "include": "seodata",
            "locale": "id-ID",
            "solId": "646b632b-435c-493b-aed5-255e56db0a05"
            # Add more parameters if required
        },
        "headers": {
            "authority": "www.jobstreet.co.id",
            "accept": "application/json, text/plain, */*",
            "accept-language": "en-US,en;q=0.9,id;q=0.8",
            "cache-control": "no-cache",
            "cookie": "ABTestID=9538b2b8-f522-46fd-ac5f-6e77927d6dba; ABSSRP=1797; ...",  # Truncated for brevity
            "pragma": "no-cache",
            "referer": "https://www.jobstreet.co.id/id/jobs?keywords=%22data%2Banalyst%22",
            "sec-ch-ua": '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"Windows"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "seek-request-brand": "jobstreet",
            "seek-request-country": "ID",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
            "x-seek-checksum": "129c8558",
            "x-seek-site": "Chalice"
            # Add more headers if required
        },
       "structure": {
            "total_jobs_key": "totalCount",
            "number_of_pages_key": "totalPages",
            "limit_key": 1,
            "offset_key": 0
        }
    },
    {
        "name": "Kalibrr",
        "url": "https://www.kalibrr.com/kjs/job_board/search",
        "params": {
            "limit": 1,
            "offset": 0,
            "country": "Indonesia",
            "text": "\"data+analyst\"",
            #"sort_direction": "desc",
            #"sort_field": "activation_date"
        },
        "headers": {
              # Not explicitly mentioned, but often present in headers
        },
        "structure": {
            "total_jobs_key": "count",
            "number_of_pages_key": 1,
            "limit_key": 1,
            "offset_key": 0
        }
    }
    # Add more job portal configurations if needed...
]

# Function to make an API request to job portals
def make_api_request(portal):
    try:
        response = requests.get(portal["url"], headers=portal.get("headers"), params=portal.get("params"))
        return response
    except requests.RequestException as e:
        print(f"Request to {portal['name']} failed:", str(e))
        return None
# Function to check connection to job portals and retrieve total job records
def check_connection_and_get_total_jobs(job_portals):
    for portal in job_portals:
        response = make_api_request(portal)
        print(f"{portal['name']} Response Status Code:", response.status_code if response else "Failed")

        # Check if the response is in JSON format
        try:
            response_json = response.json()
            file_name = f"{portal['name']}_response.json"
            with open(file_name, 'w') as file:
                json.dump(response_json, file, indent=4)
                print(f"Response for {portal['name']} saved as {file_name}")

            # Retrieve total job records if "total_jobs_key" is configured
            total_jobs_key = portal.get("structure", {}).get("total_jobs_key")
            if total_jobs_key and total_jobs_key in response_json:
                total_jobs = response_json[total_jobs_key]
                print(f"Total jobs for {portal['name']}: {total_jobs}")
            else:
                print(f"'total_jobs_key' not found or not configured for {portal['name']}")

        except json.JSONDecodeError:
            print(f"Response for {portal['name']} is not in JSON format, not saving.")
            pass  # Handle non-JSON responses (if needed)

if __name__ == "__main__":
    check_connection_and_get_total_jobs(job_portals)

Glints Response Status Code: Failed
Response for Glints saved as Glints_response.json
'total_jobs_key' not found or not configured for Glints
Jobstreet Response Status Code: 200
Response for Jobstreet saved as Jobstreet_response.json
Total jobs for Jobstreet: 126
Kalibrr Response Status Code: 200
Response for Kalibrr saved as Kalibrr_response.json
Total jobs for Kalibrr: 406
